In [1]:
# For data processing
import numpy as np
from math import sqrt
# For data processing and manipulation
import pandas as pd
import csv
 
# For date calculations
import datetime
import time
 
# For ploting data
import IPython
import IPython.display
 
import itertools
from itertools import cycle
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
 
# For checking path
import os , gc
from os import path
import csv
import json
import requests

In [2]:
from pymongo import MongoClient
import time
# pprint library is used to make the output look more pretty
'''
Should move db code to other file
'''
# connect to MongoDB, change the << MONGODB URL >> to reflect your own connection string
client = MongoClient(port=27017)
db = client.test
print(db.list_collection_names()) 

['crime.chicago', 'weather.chicago', 'crime.sf', 'weather.sf']


<h2><b> join weathers by date, pressure difference </h2></b>

In [5]:

start_time = time.time()

# MongoDB aggregation pipeline
pipeline = [
    {
        "$lookup": {
            "from": "weather.sf",
            "localField": "valid_date_time",
            "foreignField": "valid_date_time",
            "as": "sf_data",
        },
    },
    {
        "$unwind": "$sf_data",
    },
    {
        "$project": {
            "chicago_temp": "$temp",
            "chicago_pressure": "$pressure",
            "sf_temp": "$sf_data.temp",
            "sf_pressure": "$sf_data.pressure",
            "valid_date_time": 1,
        },
    },
    {
        "$addFields": {
            "temp_difference": {
                "$subtract": ["$chicago_temp", "$sf_temp"]
            },
            "pressure_difference": {
                "$subtract": ["$chicago_pressure", "$sf_pressure"]
            }
        }
    }
]

# Execute the aggregation pipeline
data = db.weather.chicago.aggregate(pipeline)
print(data)
# Collect the results into a list of dictionaries
#results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
#df = pd.DataFrame(results)

# Print the DataFrame as a table
#display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")


Execution Time: 7.97 seconds


In [8]:

start_time = time.time()

# MongoDB aggregation pipeline
pipeline = [
    {
        "$addFields": {
            "formatted_datetime": {
                "$dateFromString": {
                    "dateString": {
                        "$concat": ["$Date", " ", "$Time"]
                    },
                    "format": "%m/%d/%Y %H:%M" # Ensure this matches the format of 'Date' and 'Time' in 'crime.sf'
                }
            }
        }
    },
    {
        "$lookup": {
            "from": "crime.chicago",
            "let": {"formatted_datetime": "$formatted_datetime"}, # Corrected variable name
            "pipeline": [
                {
                    "$addFields": {
                        "_date": {
                            "$dateFromString": {
                                "dateString": "$Date",
                                "format": "%m/%d/%Y %I:%M:%S %p" # Corrected format to match 'crime.chicago'
                            }
                        }
                    }
                },
                {
                    "$match": {
                        "$expr": {
                            "$eq": ["$_date", "$$formatted_datetime"] # Corrected variable reference
                        }
                    }
                }
            ],
            "as": "joined_data"
        }
    },
    {
        "$unwind": "$joined_data"
    },
    {
        "$project": {
            "chicago_Primary_Type": "$joined_data.Primary Type", # Corrected to reference joined data
            "sf_Category": "$Category", # Assuming 'Category' exists in 'crime.sf' and corrected typo
            # Add other fields as needed
        }
    }
]
# Execute the aggregation pipeline
data = db.crime.chicago.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")


""


Execution Time: 115.47 seconds
